# Imports

In [122]:
import numpy as np
import pandas as pd

import plotly.express as px

# Load in Data

In [123]:
pickle_to_load = "../../FantasyData/data-frames/df_wr_college_2000_2023.pkl"

In [124]:
df_pos = pd.read_pickle(pickle_to_load)

### For saving Data Later

In [125]:
save_path = "../../interactive-2.0/WR/draft-capital/"

# Clean Up Data

In [126]:
df_clean = df_pos.copy().dropna(subset=["draft_ovr"])

# Get Draft Capital Success Rate By Round

Grab the best (lower tier is better) tier a player has been in, for each round of the draft:

In [127]:
tier_counts = df_clean.groupby(['draft_round', 'player_name'])['tier'].min().reset_index()

Then group this data frame by the draft round to get the total number of players in each round

In [128]:
total_players_per_round = tier_counts \
    .groupby('draft_round')['player_name'] \
    .count() \
    .reset_index(name='total_players')

Grab the number of each players in each tier, by their draft round:

In [129]:
players_in_tier = tier_counts.groupby(['draft_round', 'tier']).size().reset_index(name='players_in_tier')

Merge the dataframes together:

In [130]:
merged_data = pd.merge(players_in_tier, total_players_per_round, on='draft_round')
merged_data['percentage'] = (merged_data['players_in_tier'] / merged_data['total_players']) * 100

Tier percentagers by draft round:

In [131]:
merged_data.head(5)

,draft_round,tier,players_in_tier,total_players,percentage
0,1.0,1,30,74,40.540541
1,1.0,2,11,74,14.864865
2,1.0,3,11,74,14.864865
3,1.0,4,4,74,5.405405
4,1.0,5,3,74,4.054054


# Analysis

In [132]:
df_success = merged_data.query("tier <= 4").groupby("draft_round").apply(lambda x: x)

/tmp/ipykernel_9026/3608773231.py:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [133]:
df_success.drop(columns="draft_round", inplace=True)

In [134]:
df_success.reset_index(inplace=True)

In [135]:
df_success.rename(columns={"draft_round": "Draft Round", 
                           "percentage": "Success Rate", "players_in_tier": "Sample Size of Tier",
                           "total_players": "All Players in this Round",
                           "tier": "Best Season Tier"
                          }, inplace=True)

In [136]:
df_success.drop(columns="level_1", inplace=True)

In [137]:
df_success.head()

,Draft Round,Best Season Tier,Sample Size of Tier,All Players in this Round,Success Rate
0,1.0,1,30,74,40.540541
1,1.0,2,11,74,14.864865
2,1.0,3,11,74,14.864865
3,1.0,4,4,74,5.405405
4,2.0,1,21,85,24.705882


### Bust DF

In [138]:
df_bust = merged_data.query("tier > 4").groupby("draft_round").agg({"percentage": "sum"})

In [139]:
df_bust.rename(columns={"percentage": "Chance of Bust"}, inplace=True)

In [140]:
df_bust = df_bust.reset_index().rename(columns={'draft_round': 'Draft Round'})

In [144]:
df_bust["Chance of Success"] = 100 - df_bust["Chance of Bust"]

In [145]:
df_bust

,Draft Round,Chance of Bust,Chance of Success
0,1.0,24.324324,75.675676
1,2.0,36.470588,63.529412
2,3.0,54.166667,45.833333
3,4.0,75.000000,25.000000
4,5.0,70.833333,29.166667
5,6.0,87.272727,12.727273
6,7.0,90.697674,9.302326


# Highlighting

In [146]:
# Define a function to highlight rows based on a condition 
def highlight_row(row): 
    # Mapping of pick ranges to background colors
    color_map = {
        1.0: 'yellow',
        2.0: 'lightblue',
        3.0: 'lightgreen',
        4.0: 'lightpink',
        5.0: 'lightgrey',
        6.0: 'lightcoral',
        7.0: 'lightsalmon',
    }
    
    # Get the background color for the current row's pick range
    # Default to white if the pick range is not in the map
    bg_color = color_map.get(row['Draft Round'], 'white')
    
    # Return a list of styles for each column in the row
    return [f'background-color: {bg_color}']*len(row)

In [147]:
df_1 = df_success.style.apply(highlight_row, axis=1)

In [148]:
df_1

,Draft Round,Best Season Tier,Sample Size of Tier,All Players in this Round,Success Rate
0,1.000000,1,30,74,40.540541
1,1.000000,2,11,74,14.864865
2,1.000000,3,11,74,14.864865
3,1.000000,4,4,74,5.405405
4,2.000000,1,21,85,24.705882
5,2.000000,2,12,85,14.117647
6,2.000000,3,10,85,11.764706
7,2.000000,4,11,85,12.941176
8,3.000000,1,12,72,16.666667
9,3.000000,2,9,72,12.500000


In [149]:
# with open(f'{save_path}/success-by-round-table.html', 'w') as file:
#     file.write(df_1.to_html())

In [150]:
df_2 = df_bust.style.apply(highlight_row, axis=1)

In [151]:
df_2

,Draft Round,Chance of Bust,Chance of Success
0,1.000000,24.324324,75.675676
1,2.000000,36.470588,63.529412
2,3.000000,54.166667,45.833333
3,4.000000,75.000000,25.000000
4,5.000000,70.833333,29.166667
5,6.000000,87.272727,12.727273
6,7.000000,90.697674,9.302326


In [152]:
with open(f'{save_path}/busts-by-round-table.html', 'w') as file:
    file.write(df_2.to_html())

# Plotly Bar Charts

In [89]:
df_success["Best Season Tier"] = df_success["Best Season Tier"].astype(float)

In [90]:
fig = px.bar(df_success, x="Draft Round", y="Success Rate",
             color='Best Season Tier', barmode='group',
             hover_data=['Sample Size of Tier', 'All Players in this Round'],
             )

fig.write_html(f"{save_path}/hit-rate-v-round-alternative.html")
fig.show()

In [85]:
df_success["Draft Round"] = df_success["Draft Round"].astype(float)

In [87]:
fig = px.bar(df_success, x="Best Season Tier", y="Success Rate",
             color='Draft Round', barmode='group',
             hover_data=['Sample Size of Tier', 'All Players in this Round'],
             )

fig.write_html(f"{save_path}/hit-rate-v-tier-by-round-alternative.html")
fig.show()

In [78]:
fig = px.bar(df_bust, x="Draft Round", y="Chance of Bust")

fig.write_html(f"{save_path}/bust-rate-v-round.html")
fig.show()